In [1]:
import urllib.request
import json
import pandas as pd
import plotly.express as px

In [2]:
#0501013B0 = Amoxicillin

url_0501013B0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501013B0&format=json"
response_0501013B0 = urllib.request.urlopen(url_0501013B0)
data_0501013B0 = json.loads(response_0501013B0.read())
flat_data_0501013B0 = pd.json_normalize(data_0501013B0)
flat_data_0501013B0 = flat_data_0501013B0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501013B0 = flat_data_0501013B0.drop(columns=['items', 'quantity'])
flat_data_0501013B0 .rename(columns={'actual_cost': 'Amoxicillin'}, inplace=True)
flat_data_0501013B0

Amoxicillin
row_name         row_id date                   
NHS BARNSLEY CCG 02P    2016-05-01      4077.60
                        2016-06-01      3371.07
                        2016-07-01      3168.21
                        2016-08-01      2574.71
                        2016-09-01      3019.71
...                                         ...
NHS WIRRAL CCG   12F    2020-12-01      4516.10
                        2021-01-01      3984.33
                        2021-02-01      3270.02
                        2021-03-01      3681.28
                        2021-04-01      4233.48

[6382 rows x 1 columns]

In [3]:
#0501030I0 = Doxycycline Hyclate

url_0501030I0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501030I0&format=json"
response_0501030I0 = urllib.request.urlopen(url_0501030I0)
data_0501030I0 = json.loads(response_0501030I0.read())
flat_data_0501030I0 = pd.json_normalize(data_0501030I0)
flat_data_0501030I0 = flat_data_0501030I0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501030I0 = flat_data_0501030I0.drop(columns=['items', 'quantity'])
flat_data_0501030I0.rename(columns={'actual_cost': 'Doxycycline Hyclate'}, inplace=True)
flat_data_0501030I0

Doxycycline Hyclate
row_name         row_id date                           
NHS BARNSLEY CCG 02P    2016-05-01              2185.46
                        2016-06-01              2129.81
                        2016-07-01              1781.57
                        2016-08-01              1839.70
                        2016-09-01              1997.36
...                                                 ...
NHS WIRRAL CCG   12F    2020-12-01              3539.57
                        2021-01-01              4467.21
                        2021-02-01              3338.46
                        2021-03-01              3777.32
                        2021-04-01              3562.62

[6382 rows x 1 columns]

In [4]:
#0501021L0 = Cefalexin



url_0501021L0 = "https://openprescribing.net/api/1.0/spending_by_ccg/?code=0501021L0&format=json"
response_0501021L0 = urllib.request.urlopen(url_0501021L0)
data_0501021L0 = json.loads(response_0501021L0.read())
flat_data_0501021L0= pd.json_normalize(data_0501021L0)
flat_data_0501021L0 = flat_data_0501021L0.groupby(['row_name', 'row_id', 'date']).sum()
flat_data_0501021L0 = flat_data_0501021L0.drop(columns=['items', 'quantity'])
flat_data_0501021L0.rename(columns={'actual_cost': 'Cefalexin'}, inplace=True)
flat_data_0501021L0

Cefalexin
row_name         row_id date                 
NHS BARNSLEY CCG 02P    2016-05-01     289.58
                        2016-06-01     237.11
                        2016-07-01     275.00
                        2016-08-01     221.24
                        2016-09-01     268.23
...                                       ...
NHS WIRRAL CCG   12F    2020-12-01    1989.70
                        2021-01-01    2071.17
                        2021-02-01    1874.03
                        2021-03-01    2210.04
                        2021-04-01    2152.52

[6375 rows x 1 columns]

In [5]:
join_1 = flat_data_0501013B0.join(flat_data_0501030I0, lsuffix='row_id', rsuffix='row_id')
all_amoxicillin_merged = join_1.join(flat_data_0501021L0, lsuffix='row_id', rsuffix='row_id')
all_amoxicillin_merged.fillna(0, inplace=True)
all_amoxicillin_merged['Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin(£)']= all_amoxicillin_merged.iloc[:, -3:].sum(axis=1)
all_amoxicillin = all_amoxicillin_merged.reset_index()
all_amoxicillin.rename(columns={'row_name': 'Clinical Commissioning Group (CCG)', 'row_id': 'ODS code', 'date': 'Date'}, inplace=True)
all_amoxicillin 

,Clinical Commissioning Group (CCG),ODS code,Date,Amoxicillin,Doxycycline Hyclate,Cefalexin,"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin(£)"
0,NHS BARNSLEY CCG,02P,2016-05-01,4077.60,2185.46,289.58,6552.64
1,NHS BARNSLEY CCG,02P,2016-06-01,3371.07,2129.81,237.11,5737.99
2,NHS BARNSLEY CCG,02P,2016-07-01,3168.21,1781.57,275.00,5224.78
3,NHS BARNSLEY CCG,02P,2016-08-01,2574.71,1839.70,221.24,4635.65
4,NHS BARNSLEY CCG,02P,2016-09-01,3019.71,1997.36,268.23,5285.30
...,...,...,...,...,...,...,...
6377,NHS WIRRAL CCG,12F,2020-12-01,4516.10,3539.57,1989.70,10045.37
6378,NHS WIRRAL CCG,12F,2021-01-01,3984.33,4467.21,2071.17,10522.71
6379,NHS WIRRAL CCG,12F,2021-02-01,3270.02,3338.46,1874.03,8482.51
6380,NHS WIRRAL CCG,12F,2021-03-01,3681.28,3777.32,2210.04,9668.64


In [6]:
all_amoxicillin_plot = all_amoxicillin.groupby(['Date']).sum()
all_amoxicillin_plot = all_amoxicillin_plot.reset_index()

In [7]:
all_amoxicillin_map = all_amoxicillin.groupby(['ODS code']).sum()
all_amoxicillin_map = all_amoxicillin_map.reset_index()
all_amoxicillin_map

,ODS code,Amoxicillin,Doxycycline Hyclate,Cefalexin,"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin(£)"
0,00L,274883.90,183887.82,36663.65,495435.37
1,00N,142867.96,89192.19,20321.93,252382.08
2,00P,243653.87,158080.68,61872.16,463606.71
3,00Q,176301.03,53957.06,26575.82,256833.91
4,00R,163054.15,134160.47,23817.18,321031.80
...,...,...,...,...,...
103,D9Y0V,1050167.62,846184.27,164019.93,2060371.82
104,M1J4Y,859518.01,300663.01,81680.38,1241861.40
105,M2L0M,351218.12,360969.89,49127.76,761315.77
106,W2U3Z,1586583.66,678308.47,226971.19,2491863.32


In [8]:
selected_columns = all_amoxicillin_map[["ODS code","Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin(£)"]]
all_amoxicillin_map_final = selected_columns.copy()
all_amoxicillin_map_final

,ODS code,"Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin(£)"
0,00L,495435.37
1,00N,252382.08
2,00P,463606.71
3,00Q,256833.91
4,00R,321031.80
...,...,...
103,D9Y0V,2060371.82
104,M1J4Y,1241861.40
105,M2L0M,761315.77
106,W2U3Z,2491863.32


In [9]:
pd.options.plotting.backend = "plotly"
fig = px.bar(all_amoxicillin_plot, x='Date', y= ["Amoxicillin", "Doxycycline Hyclate", 'Cefalexin'],
color_discrete_sequence=["#003087", "#0072CE", "#41B6E6"],
labels={"value": "Cost (£)", "variable": "Antibiotic"},  
title= "Total cost of Amoxicillin, Doxycycline Hyclate, and Cefalexin (£) per month")
fig.update_layout(
    {"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)"},
    autosize=True,
    margin=dict(l=50, r=50, b=50, t=50, pad=4, autoexpand=True),
)
fig.show()


In [10]:
url_geo_codes = "https://openprescribing.net/api/1.0/org_location/?org_type=ccg&format=json"
response_geo_codes = urllib.request.urlopen(url_geo_codes)
data_geo_codes = json.loads(response_geo_codes.read())
#flat_data_geo_codes= pd.json_normalize(data_geo_codes)
#flat_data_geo_codes
data_geo_codes


{'type': 'FeatureCollection',
 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}},
 'features': [{'type': 'Feature',
   'properties': {'name': 'NHS NORTHUMBERLAND CCG',
    'code': '00L',
    'ons_code': None,
    'org_type': 'CCG'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[-1.618946, 55.39024],
       [-1.609637, 55.395512],
       [-1.610676, 55.397161],
       [-1.625395, 55.3977],
       [-1.612218, 55.39587],
       [-1.616184, 55.392552],
       [-1.623403, 55.389286],
       [-1.614629, 55.379927],
       [-1.608316, 55.383782],
       [-1.586855, 55.343553],
       [-1.570543, 55.339697],
       [-1.592772, 55.343188],
       [-1.610204, 55.349455],
       [-1.617773, 55.347109],
       [-1.627112, 55.348572],
       [-1.61728, 55.343946],
       [-1.613981, 55.346515],
       [-1.601686, 55.344587],
       [-1.586741, 55.336041],
       [-1.571996, 55.33856],
       [-1.549404, 55.321836],
       [-1.573121, 55.275345],
       [-1.498985, 55.185

In [21]:
fig= px.choropleth(all_amoxicillin_map_final, 
geojson = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
featureidkey= 'properties.ST_NM', 
locations= 'ODS code', 
color='Total cost of Amoxicillin, Doxycycline Hyclate, Cefalexin(£)', 
color_continuous_scale='Inferno', 
title='test', 
height=700)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()